In [1]:
from collections import defaultdict
import torch
import numpy as np




def read_graph(file_path):
    """
    Reads a weighted directed graph from a file. Each line contains three values:
    start vertex, end vertex, and edge weight.

    Args:
        file_path (str): Path to the file containing the graph.

    Returns:
        edges (list): List of tuples representing directed edges (start, end, weight).
    """
    edges = []
    try:
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line:  # Skip empty lines
                    start, end, weight = map(float, line.split())
                    edges.append((int(start), int(end), weight))
    except Exception as e:
        print(f"Error reading graph: {e}")
    return edges

def initialize_graph(edges):
    """Converts edge list to adjacency list and weights dictionary."""
    graph = defaultdict(list)
    weights = {}
    for u, v, w in edges:
        graph[u].append(v)
        weights[(u, v)] = w
    return graph, weights

def find_cycles_and_reduce(graph, weights, n):
    """Phase 1: Find cycles and reduce weights using a copy."""
    weights_copy = weights.copy()  # Work with a copy of weights
    removed_edges = set()
    removed_weights = {}

    while True:
        cycle = find_cycle(graph,  n)
        if not cycle:  # No cycle found
            break

        # Ensure all edges in the cycle exist in the weights dictionary
        cycle = [(u, v) for u, v in cycle if (u, v) in weights_copy]

        if not cycle:  # If no valid cycle exists, continue
            continue

        min_weight = min(weights_copy[(u, v)] for u, v in cycle)

        for u, v in cycle:
            weights_copy[(u, v)] -= min_weight
            if weights_copy[(u, v)] <= 0:
                # Ensure the edge is in the graph before removing
                if v in graph[u]:
                    graph[u].remove(v)
                    removed_edges.add((u, v))
                    removed_weights[(u, v)] = weights[(u, v)]

    return removed_edges, removed_weights

   





from collections import deque

def find_cycle(graph, n):
    """Detect a cycle in the graph using DFS and return the cycle as a list of edges."""
    visited = [False] * n
    stack = [False] * n
    parent = [-1] * n

    def dfs(v):
        visited[v] = True
        stack[v] = True
        for neighbor in graph[v]:
            if not visited[neighbor]:
                parent[neighbor] = v
                cycle = dfs(neighbor)
                if cycle:
                    return cycle
            elif stack[neighbor]:
                # Found a cycle, reconstruct it
                cycle = []
                current = v
                while current != neighbor:
                    cycle.append((parent[current], current))
                    current = parent[current]
                cycle.append((parent[neighbor], neighbor))
                return cycle
        stack[v] = False
        return None

    for i in range(n):
        if not visited[i]:
            cycle = dfs(i)
            if cycle:
                return cycle
    return None

from collections import deque

import numpy as np

def find_minimum_weight_cycle(graph, weights, n):
    """
    Find the minimum weight cycle in the graph using Floyd-Warshall.

    :param graph: Adjacency list representation of the graph.
    :param weights: Dictionary of edge weights.
    :param n: Total number of vertices in the graph.
    :return: List of edges representing the minimum weight cycle.
    """
    # Step 1: Initialize distance and predecessor matrices
    dist = np.full((n, n), float('inf'))
    pred = [[-1 for _ in range(n)] for _ in range(n)]

    # Fill in the distances based on edge weights
    for u in range(n):
        dist[u][u] = 0
        if u in graph:
            for v in graph[u]:
                dist[u][v] = weights.get((u, v), float('inf'))
                pred[u][v] = u

    # Step 2: Run Floyd-Warshall algorithm
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]

    # Step 3: Find the minimum weight cycle
    min_cycle_weight = float('inf')
    cycle = []

    for u in range(n):
        for v in range(n):
            if u != v and dist[u][v] < float('inf') and dist[v][u] < float('inf'):
                cycle_weight = dist[u][v] + dist[v][u]
                if cycle_weight < min_cycle_weight:
                    min_cycle_weight = cycle_weight
                    # Reconstruct the cycle
                    cycle = []
                    # Trace path from u to v
                    current = v
                    while current != u:
                        cycle.append((pred[u][current], current))
                        current = pred[u][current]
                    # Trace path from v back to u
                    current = u
                    while current != v:
                        cycle.append((pred[v][current], current))
                        current = pred[v][current]

    # Return the minimum weight cycle if found
    if min_cycle_weight == float('inf'):
        return None
    else:
        return cycle





def check_and_readd_edges(graph, removed_edges, n):
    """Phase 2: Check and re-add edges if they do not create a cycle."""
    
    def has_path(start, end, graph):
        """Helper function to check if there is a path from start to end using DFS."""
        visited = [False] * n
        stack = [start]
        while stack:
            node = stack.pop()
            if node == end:
                return True
            if not visited[node]:
                visited[node] = True
                stack.extend(graph[node])
        return False

    readded_edges = set()
    f = sorted(list(removed_edges), reverse=True)

    for u, v in f:
        if not has_path(v, u, graph):  # Only re-add if it doesn't create a cycle
            graph[u].append(v)
            readded_edges.add((u, v))

    return removed_edges - readded_edges

class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # Path compression
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)
        if root_x != root_y:
            if self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            elif self.rank[root_x] < self.rank[root_y]:
                self.parent[root_y] = root_y
            else:
                self.parent[root_y] = root_x
                self.rank[root_x] += 1

def mwfas(file_path):
    """
    Main function to find Minimum Weighted Feedback Arc Set (MWFAS).
    :param file_path: Path to the file containing the graph.
    :return: A dictionary with metrics, updated graph, removed edges, and their weights.
    """
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _ in edges) + 1
    graph, weights = initialize_graph(edges)

    # Original graph statistics
    total_edges = len(edges)
    total_weight = sum(w for _, _, w in edges)

    # Phase 1: Reduce cycles
  #  print("Before Phase 1:")
  #  print("Graph:", graph)
  #  print("Weights:", weights)
    
    removed_edges, removed_weights = find_cycles_and_reduce(graph, weights, n)
    
   # print("After Phase 1:")
   # print("Removed edges:", removed_edges)
   # print("Removed weights:", removed_weights)

    # Phase 2: Re-add edges (if applicable)
    removed_edges = check_and_readd_edges(graph, removed_edges, n)

    # Compute final metrics
    num_removed_edges = len(removed_edges)
    total_removed_weight = sum(removed_weights.get(edge, 0) for edge in removed_edges)

#    print("Final Results:")
#    print("Removed edges:", removed_edges)
#    print("Weights of removed edges:", {edge: removed_weights.get(edge, 0) for edge in removed_edges})
#    print("Final graph:", graph)

    # Return results
    return {
        "total_edges": total_edges,
        "total_weight": total_weight,
        "num_removed_edges": num_removed_edges,
        "removed_weight": total_removed_weight,
        "final_graph": graph,
        "removed_edges": removed_edges,
        "removed_weights": {edge: removed_weights.get(edge, 0) for edge in removed_edges},
    }

    

# Example usage




In [15]:
# After feedback arc set removal
def compute_vertex_rankings(graph, weights, n):
 #   print("in the input dag to topol sort ", "graph is ",graph, "and weights are ",weights)
    """
    Compute rankings for the vertices in a DAG.
    :param graph: Adjacency list of the DAG.
    :param weights: Dictionary of edge weights.
    :param n: Total number of vertices in the graph.
    :return: A list of rankings for the vertices.
    """
    # Step 1: Calculate in-degrees
    in_degree = [0] * n
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    # Step 2: Perform topological sort using a min-heap
    from heapq import heappop, heappush
    min_heap = []
    for i in range(n):
        if in_degree[i] == 0:
            heappush(min_heap, i)

    topological_order = []
    while min_heap:
        current = heappop(min_heap)
        topological_order.append(current)
        for neighbor in graph[current]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                heappush(min_heap, neighbor)

    # Step 3: Calculate outgoing edge weight sums for all vertices
    outgoing_weights = {v: 0 for v in range(n)}
    incoming_weights = {v: 0 for v in range(n)}

    for u in graph:
        for v in graph[u]:
            outgoing_weights[u] += weights.get((u, v), 0)
            incoming_weights[v] += weights.get((u, v), 0)


    # Step 4: Assign rankings
    rankings = [-1] * n
    current_rank = 0
    for vertex in topological_order:
        rankings[vertex] = current_rank
        current_rank += 1

    # Break ties for vertices with the same ranking based on outgoing edge weights
    tied_vertices = sorted(
        [(rankings[v], -(outgoing_weights[v] - incoming_weights[v]) / 
          (outgoing_weights[v] + incoming_weights[v] if outgoing_weights[v] + incoming_weights[v] > 0 else 1), v)
         for v in range(n)],
        key=lambda x: (x[0], x[1])
    )

    scores = [0] * n
    for final_rank, (_, _, vertex) in enumerate(tied_vertices):
        scores[vertex]=-final_rank

    print(scores)
    return scores

# Ensure graph is a DAG after feedback arc set removal



In [16]:
def optimize_updated_scores(adjacency_matrix, scores, epsilon=1e-8):
    import cvxpy as cp
    import numpy as np

    n = len(scores)

    # Convert inputs to numpy
    adjacency_matrix_np = adjacency_matrix.numpy()
    scores_np = scores.numpy().flatten()

    # Define optimization variables
    updated_scores = cp.Variable(n)
    R = cp.Variable((n, n))  # Auxiliary variable for ratios

    # Compute M (skew-symmetric pairwise matrix)
    M = adjacency_matrix_np - adjacency_matrix_np.T

    # Edge mask
    edge_mask = (adjacency_matrix_np + adjacency_matrix_np.T > 0).astype(float)

    # Objective: Minimize squared difference between M and R
    objective = cp.sum_squares(cp.multiply(edge_mask, M - R))

    # Constraints for R_ij
    constraints = []
    for i in range(n):
        for j in range(n):
            if edge_mask[i, j] > 0:
                constraints.append(R[i, j] * (updated_scores[i] + updated_scores[j] + epsilon) == updated_scores[i] - updated_scores[j])

    # Order-preserving constraints
    constraints += [
        updated_scores[i] <= updated_scores[j]
        for i in range(n)
        for j in range(n)
        if scores_np[i] <= scores_np[j]
    ]

    # Solve the problem
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()

    return updated_scores.value


In [17]:
def graph_to_adjacency_matrix(graph, weights, n):
    """
    Converts a graph represented as an adjacency list with weights to an adjacency matrix.

    :param graph: Adjacency list of the graph.
    :param weights: Dictionary of edge weights where the key is a tuple (u, v) and the value is the weight.
    :param n: Number of vertices in the graph.
    :return: A torch.FloatTensor adjacency matrix with weights.
    """
    adjacency_matrix = torch.zeros((n, n))
    for u in graph:
        for v in graph[u]:
            adjacency_matrix[u, v] = weights[(u, v)] # Weighted edge from u to v
  #  print("adjacency matrix= ",adjacency_matrix)
 #   print(adjacency_matrix)
    return adjacency_matrix




In [18]:
import torch
import numpy as np

def reorder_floats(x):
    n = len(x)
    random_floats = np.random.uniform(0,  2*n/3, n)
    y = np.zeros(n)
    for idx, val in enumerate(np.argsort(x)):
        y[val] = sorted(random_floats)[idx]
    return y

def calculate_upset_loss(adjacency_matrix, scores, style='ratio', margin=0.01):
    """
    Calculate the upset loss for the graph rankings using adjacency matrix and scores.

    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param scores: Torch FloatTensor ranking scores (n x 1).
    :param style: Type of upset loss ('naive', 'simple', 'ratio', or 'margin').
    :param margin: Margin for margin loss (default: 0.01).
    :return: Torch FloatTensor upset loss value.
    """
    epsilon = 1e-8  # For numerical stability

    # Ensure scores are 2D
    if scores.ndim == 1:
        scores = scores.view(-1, 1)

    # Skew-symmetric pairwise comparison matrix (M)
    M1 = adjacency_matrix - adjacency_matrix.T

    # Normalize scores to [0, 1] range
    normalized_scores = scores

    # Pairwise score differences (T)
    T1 = normalized_scores - normalized_scores.T

    # Edge mask: Only consider meaningful edges (where M != 0)
    edge_mask = M1 != 0

    if style == 'ratio':
        min_upset = float('inf')  # Initialize with a large value
        
        for _ in range(40):
            # Generate reordered scores using reorder_floats
            if _==0:
                reordered_scores=scores
            else:
                reordered_scores = torch.FloatTensor(reorder_floats(scores.flatten().tolist()))
            reordered_scores = reordered_scores.view(-1, 1)

            # Compute T2 for normalized scores
            T2 = reordered_scores + reordered_scores.T + epsilon
            T = torch.div(T1, T2)
            M2 = adjacency_matrix + adjacency_matrix.T + epsilon
            M3 = torch.div(M1, M2)  # Normalize the adjacency matrix
            
            # Compute ratio-based upset loss for this iteration
            powers = torch.pow((M3 - T)[edge_mask], 2)
            upset_loss = torch.sum(powers) / torch.sum(edge_mask)

            # Track the minimum upset loss
            min_upset = min(min_upset, upset_loss.item())
        
        return torch.tensor(min_upset)

    elif style == 'naive':
        upset = torch.sum(torch.sign(T1[edge_mask]) != torch.sign(M1[edge_mask])) / torch.sum(edge_mask)

    elif style == 'simple':
        upset = torch.mean((torch.sign(T1[edge_mask]) - torch.sign(M1[edge_mask]))**2)

    elif style == 'margin':
        upset = torch.mean(torch.nn.functional.relu(-M1[edge_mask] * (T1[edge_mask] - margin)))

    else:
        raise ValueError(f"Unsupported style: {style}")

    return upset


In [19]:
import torch

def compute_ratio_upset_loss(adjacency_matrix, scores, epsilon=1e-8):
    """
    Compute the ratio upset loss for the graph rankings using adjacency matrix and scores.

    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param scores: Torch FloatTensor ranking scores (n x 1).
    :param epsilon: Small value for numerical stability (default: 1e-8).
    :return: Torch FloatTensor ratio upset loss value.
    """
    # Ensure scores are 2D
    if scores.ndim == 1:
        scores = scores.view(-1, 1)

    # Skew-symmetric pairwise comparison matrix (M)
    M1 = adjacency_matrix - adjacency_matrix.T

    # Pairwise score differences (T1)
    T1 = scores - scores.T

    # Edge mask: Only consider meaningful edges (where M1 != 0)
    edge_mask = M1 != 0

    # Compute T2 for normalized scores
    T2 = scores + scores.T + epsilon
    T = torch.div(T1, T2)

    # Normalize M1 using adjacency matrix
    M2 = adjacency_matrix + adjacency_matrix.T + epsilon
    M3 = torch.div(M1, M2)  # Normalize the adjacency matrix

    # Compute ratio upset loss
    powers = torch.pow((M3 - T)[edge_mask], 2)
    upset_loss = torch.sum(powers) / torch.sum(edge_mask)

    return upset_loss


In [20]:
import torch
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

def minimize_ratio_upset_loss(adjacency_matrix, scores, epsilon=1e-8):
    """
    Find the scores that minimize the ratio upset loss for a given adjacency matrix.

    :param adjacency_matrix: Torch FloatTensor adjacency matrix (n x n).
    :param epsilon: Small value for numerical stability (default: 1e-8).
    :return: Tuple (optimal_scores, minimized_loss)
    """
    n = adjacency_matrix.shape[0]

    # Objective function to minimize
    def objective_function(scores):
        # Convert scores to Torch FloatTensor
        scores = torch.tensor(scores, dtype=torch.float32)
        # Compute ratio upset loss
        return compute_ratio_upset_loss(adjacency_matrix, scores, epsilon)

    # Initial guess for scores (uniform values)
    initial_guess = scores

    # Bounds to ensure scores remain positive
    bounds = [(1e-6, None)] * n  # Lower bound is 1e-6 to avoid division by zero

    # Minimize the objective function
    result = differential_evolution(
    objective_function,
    bounds=bounds,
    strategy='best1bin',
    maxiter=1000,
    tol=1e-9
)

    if not result.success:
        raise ValueError(f"Optimization failed: {result.message}")

    # Optimal scores and minimized loss
    optimal_scores = result.x
    minimized_loss = result.fun

    return optimal_scores, minimized_loss

# Example usage
# adjacency_matrix = torch.FloatTensor([
#     [0, 1, 1, 0],
#     [1, 0, 1, 1],
#     [1, 1, 0, 1],
#     [0, 1, 1, 0]
# ])

# optimal_scores, minimized_loss = minimize_ratio_upset_loss(adjacency_matrix)
# print("Optimal Scores:", optimal_scores)
# print("Minimized Ratio Upset Loss:", minimized_loss)


In [21]:
def evaluate_upset_losses(file_path, rankings):
    """
    Evaluate upset losses (naive, simple, ratio, margin) for a graph and given rankings.

    :param file_path: Path to the graph file.
    :param rankings: List of rankings for the vertices.
    """
    # Step 1: Prepare Graph and Adjacency Matrix
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _ in edges) + 1
    graph, weights = initialize_graph(edges)
    adjacency_matrix = graph_to_adjacency_matrix(graph, weights, n)

    # Step 2: Convert Rankings to Scores Tensor
    scores = torch.FloatTensor(rankings).view(-1, 1)

    # Step 3: Calculate Upset Losses
    naive_loss = calculate_upset_loss(adjacency_matrix, scores, style='naive').item()
    simple_loss = calculate_upset_loss(adjacency_matrix, scores, style='simple').item()
    ratio_loss = calculate_upset_loss(adjacency_matrix, scores, style='ratio').item()
    margin_loss = calculate_upset_loss(adjacency_matrix, scores, style='margin').item()

    # Step 4: Print Results
    print("Upset Losses for the Graph Rankings:")
    print(f"Naive Upset Loss: {naive_loss:.4f}")
    print(f"Simple Upset Loss: {simple_loss:.4f}")
    print(f"Differentiable Upset Loss (Ratio): {ratio_loss:.4f}")
    print(f"Upset Margin Loss: {margin_loss:.4f}")


In [36]:
# Main Execution
import time
import pandas as pd
import torch

# List of input files
file_paths = ["1994adj.txt"]

# Initialize results storage
results = []

# Iterate through each input file
for file_path in file_paths:
    start_time = time.time()

    # Step 1: Read the graph and initialize structures
    edges = read_graph(file_path)
    n = max(max(u, v) for u, v, _ in edges) + 1
    print(f"Processing file: {file_path} - Number of nodes: {n}, Number of edges: {len(edges)}")

    init_graph, init_weights = initialize_graph(edges)

    # Step 2: Ensure the graph is a DAG by removing cycles
    result = mwfas(file_path)
    new_graph = result['final_graph']
    new_weights = {key: value for key, value in init_weights.items() if key not in result['removed_weights']}

    # Step 3: Compute rankings for the vertices using the modified graph
    final_rankings = compute_vertex_rankings(new_graph, new_weights, n)

    # Step 4: Evaluate upset losses
    scores = torch.FloatTensor(final_rankings).view(-1, 1)
    adjacency_matrix = graph_to_adjacency_matrix(init_graph, init_weights, n)

    naive_loss = calculate_upset_loss(adjacency_matrix, scores, style='naive')  # No .item()
    simple_loss = calculate_upset_loss(adjacency_matrix, scores, style='simple')  # No .item()
    ratio_loss = calculate_upset_loss(adjacency_matrix, scores, style='ratio')  # No .item()

    end_time = time.time()
    elapsed_time = end_time - start_time
    print("naive loss=",naive_loss)
    print("simplee loss=",simple_loss)
    print("ratio loss=",ratio_loss)
    print("elapsed time=",elapsed_time)

    # Store results in a list
    # results.append({
    #     "File": "Basketball_"+str(file_path[:9]),
    #     "Nodes": n,
    #     "Edges": len(edges),
    #     "Naive Loss": round(naive_loss.item(),2),
    #     "Simple Loss": round(simple_loss.item(),2),
    #     "Ratio Loss": round(ratio_loss.item(),2),
    #     "Elapsed Time (s)": round(elapsed_time,2)
    # })
    # print
    # print("simple loss=",simple_loss.item())

# Write results to an Excel file
#output_file = "graph_analysis_results.xlsx"
#df = pd.DataFrame(results)
#df.to_excel(output_file, index=False)

#print(f"Results written to {output_file}")




Processing file: 1994adj.txt - Number of nodes: 301, Number of edges: 3144
[-60, -97, -184, -26, -41, -27, 0, -201, -273, -24, -1, -19, -271, -248, -108, -243, -226, -65, -299, -121, -185, -63, -133, -113, -5, -52, -148, -269, -118, -160, -109, -31, -190, -83, -112, -2, -38, -32, -59, -151, -56, -181, -163, -149, -45, -78, -40, -3, -101, -93, -265, -15, -223, -285, -170, -48, -176, -283, -263, -241, -39, -95, -229, -272, -68, -25, -128, -164, -136, -57, -77, -46, -114, -111, -188, -53, -13, -123, -99, -12, -6, -250, -140, -240, -98, -30, -82, -161, -153, -80, -150, -107, -47, -49, -130, -4, -7, -138, -100, -67, -8, -175, -35, -192, -200, -252, -85, -55, -84, -139, -74, -152, -116, -274, -66, -54, -137, -132, -86, -179, -205, -76, -159, -58, -105, -242, -22, -9, -258, -194, -228, -64, -10, -28, -61, -122, -262, -251, -62, -36, -102, -33, -11, -69, -220, -154, -75, -50, -42, -79, -18, -14, -212, -43, -141, -203, -214, -115, -117, -17, -16, -253, -177, -234, -70, -182, -279, -120, -224, -